In [19]:
states_file = "stateabbr.txt"
# removed from state things that I don't need and also changed Alabama with Alberta al and ab.
with open(states_file, 'r', encoding='latin-1') as input_file:
    data = input_file.readlines()
    # I removed the Us States, and Canada from original file
    # if line.strip() removes the blank lines and line.rstrip removes the \n character.
    lines = [line.rstrip('\n') for line in data if line.strip()]
    split_lines = [x.split(" ")[0] for x in lines]
    #print(split_lines)
    print(split_lines)
# adding those 2 elements because they are not on states txt.
split_lines.append("pe")
split_lines.append("gl")

['al', 'ak', 'ar', 'az', 'ca', 'co', 'ct', 'de', 'dc', 'fl', 'ga', 'hi', 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma', 'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny', 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'vt', 'va', 'vi', 'wa', 'wv', 'wi', 'wy', 'ab', 'bc', 'mb', 'nb', 'lb', 'nf', 'nt', 'ns', 'nu', 'on', 'Prince', 'qc', 'sk', 'yt', 'dengl', 'fraspm']


In [20]:
import pandas as pd
import numpy as np
# read the plants.data file to get the info about the plants, also drop the first column which is the names of the plants bcs we dont need.
df = pd.read_csv('plants.data', sep = ",", header=None, error_bad_lines=False, encoding = "ISO-8859-1", names = [x for x in range(70)]).drop(columns=[0])
df.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69
0,fl,nc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fl,nc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ct,dc,fl,hi,il,ky,la,md,mi,ms,nc,sc,va,pr,vi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ct,dc,fl,il,ky,la,md,mi,ms,nc,sc,va,pr,vi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hi,pr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # I need also a dictionary to translate codes into column numbers, with -1 for NaN (these values will be omitted):
# codeToInd = {code: ind for ind, code in enumerate(split_lines)}
# codeToInd[np.nan] = -1

# print(df.applymap(lambda x: codeToInd.get(x)))
# # The first step of actual computation is to translate df into ind - a Numpy array:
# ind = np.vectorize(codeToInd.get)(df)
# findVal = Final_Df.values


In [3]:
import time
start = time.time()

# Use get_dummies with max values per columns (for always 0,1) values and then 
# DataFrame.reindex for change order of columns and also add some non exist columns filled by 0:

df = (pd.get_dummies(df, prefix='', prefix_sep='')
        .max(axis=1, level=0)
        .reindex(split_lines, axis=1, fill_value=0))
print (df)
print("time needed: ",time.time() - start )

       al  ak  ar  az  ca  co  ct  ...  qc  sk  yt  dengl  fraspm  pe  gl
0       0   0   0   0   0   0   0  ...   0   0   0      0       0   0   0
1       0   0   0   0   0   0   0  ...   0   0   0      0       0   0   0
2       0   0   0   0   0   0   1  ...   0   0   0      0       0   0   0
3       0   0   0   0   0   0   1  ...   0   0   0      0       0   0   0
4       0   0   0   0   0   0   0  ...   0   0   0      0       0   0   0
...    ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..    ...     ...  ..  ..
34776   0   0   0   0   0   1   0  ...   0   0   0      0       0   0   0
34777   0   0   0   0   0   0   0  ...   0   0   0      0       0   0   0
34778   0   0   0   0   0   0   0  ...   0   0   0      0       0   0   0
34779   0   0   0   0   1   1   0  ...   0   0   0      0       0   0   0
34780   0   0   0   0   1   1   0  ...   0   0   0      0       0   0   0

[34781 rows x 71 columns]
time needed:  1.280188798904419


In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

for n_clusters in range(2,71):
    # max iterations till we stop, n_init is the times we will set new centroids.
    K_Means = KMeans(n_clusters = n_clusters, random_state = 5432, max_iter= 2000).fit(df)
    silhouette_avg = silhouette_score(df, K_Means.labels_)
    sillouete_all.append(silhouette_avg)
    davies_score = davies_bouldin_score(df,K_Means.labels_)
    db_all.append(davies_score)
    #calinski = calinski_harabasz_score(Final_Df,K_Means.labels_)
    print("For n_clusters =", 2,"The average silhouette_score is :", silhouette_avg)
    print("For n_clusters =", 2,"The davies_score is :", davies_score)
    #print("For n_clusters =", n_clusters,"The calinski score is :", calinski)

# getting the best number of clusters for the different metrics.
# ** it might be different **
max_sil = max(sillouete_all)
index_sil = sillouete_all.index(max(sillouete_all))
print("max sill: ", max_sil, " num_clusters =", index_sil + 2)
max_db = min(db_all)
index_db = db_all.index(min(db_all))
print("max sill: ", max_db, " num_clusters =", index_db + 2 )